# Entropy Examples

```yaml
Course:  DS 5001
Module:  03 Lab
Topic:   Homework
Author:  Andrew Avitabile
Date:    02 February 2024
```

## Set up and Configuration

In [1]:
#Import packages
import pandas as pd
import numpy as np

In [2]:
#Configuration
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
#Textparser
textparser = %run "../lib/textparser.py"

## Import data and tokenize

In [4]:
#Define text_file as Frankenstien
text_file = f"{data_home}/gutenberg/pg42324.txt"

In [5]:
#Use local library to clean data
import sys
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib) 
from textimporter import TextImporter
from textparser import TextParser

In [6]:
#Define OHCO
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

In [7]:
#Define patterns to split text into component parts of OHCO
clip_pats = [
    r"M\. W\. S\.",
    r"THE END."
]
chap_pat = r"^\s*(?:chapter|letter|preface)\s+[IVXLCDMivxlcdm]+"
sent_pat = r'[.?!;:]+'
token_pat = r"[\s',-]+"

In [8]:
#Tokenize using TextImporter.py
my_text = TextImporter(src_file=text_file, ohco_pats=[('chap', chap_pat, 'm')], clip_pats=clip_pats)
my_text.import_source()
my_text.parse_tokens()
my_text.extract_vocab()

Importing  C:/Users/Andre/OneDrive - University of Virginia/Course Materials/Spring 2024/DS5001/data/gutenberg/pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*(?:chapter|letter|preface)\s+[IVXLCDMivxlcdm]+
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by delimitter [.?!;:]+
Parsing OHCO level 3 token_num by delimitter [\s',-]+


In [9]:
#Extract tokens and rename the axisdf.query()
TOKEN = my_text.TOKENS
TOKEN = TOKEN.rename_axis(index={'chap_id': 'chap_num'})
TOKEN

token_str  term_str
chap_num para_num sent_num token_num                    
1        0        0        0               _To        to
                           1               Mrs       mrs
                  1        1           Saville   saville
                           2           England   england
                  2        0                 _          
...                                        ...       ...
28       82       1        10             lost      lost
                           11               in        in
                           12         darkness  darkness
                           13              and       and
                           14         distance  distance

[75721 rows x 2 columns]

In [10]:
#Extract the vocaublary
VOCAB = my_text.VOCAB

In [11]:
#Remove tokens only appear once and are fewer than 3 characters
VOCAB['modified_term_str'] = VOCAB.index
VOCAB.loc[(VOCAB.n == 1) & (VOCAB.n_chars < 3), 'modified_term_str'] = "<UNK>"
VOCAB

,n,n_chars,p,s,i,h,modified_term_str
term_str,,,,,,,
the,4197,3,0.055427,18.041696,4.173263,0.231312,the
and,2976,3,0.039302,25.443884,4.669247,0.183512,and
i,2852,1,0.037665,26.550140,4.730648,0.178178,i
of,2647,2,0.034957,28.606347,4.838263,0.169133,of
to,2101,2,0.027747,36.040457,5.171545,0.143493,to
...,...,...,...,...,...,...,...
overweigh,1,9,0.000013,75721.000000,16.208406,0.000214,overweigh
pledge,1,6,0.000013,75721.000000,16.208406,0.000214,pledge
salvation,1,9,0.000013,75721.000000,16.208406,0.000214,salvation


In [12]:
#Make this same change in the tokenized dataset
TOKEN['modified_term_str'] = TOKEN.term_str.map(VOCAB.modified_term_str)
TOKEN[TOKEN.modified_term_str == '<UNK>'].sample(5)

,,,,token_str,term_str,modified_term_str
chap_num,para_num,sent_num,token_num,,,
3,7,1,1,W,w,<UNK>
28,39,0,1,2d,2d,<UNK>
10,16,4,2,n,n,<UNK>
14,3,8,5,ne,ne,<UNK>
25,45,5,7,du,du,<UNK>


## Make NGRAMS

In [13]:
#NGRAM PREP
ngrams = 3
widx = [f"w{i}" for i in range(ngrams)]
widx

['w0', 'w1', 'w2']

In [14]:
def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    padded.index.names = ohco
    return padded

In [15]:
PADDED = token_to_padded(TOKEN, grouper=OHCO[:3], term_str='modified_term_str')

In [16]:
PADDED.head()

term_str
chap_num para_num sent_num token_num         
1        0        0        0              <s>
                           1               to
                           2              mrs
                           3             </s>
                  1        0              <s>

In [17]:
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):
    
    ohco = padded.index.names
    ngrams = padded.groupby(grouper).apply(lambda x: pd.concat([x.shift(0-i) for i in range(n)], axis=1)).reset_index(drop=True)
    ngrams.index = padded.index
    ngrams.columns = widx

    # ngrams = pd.concat([padded.shift(0-i) for i in range(n)], axis=1)
    # ngrams.index.name = 'ngram_num'
    # ngrams.columns = widx
    # ngrams = ngrams.fillna('<EOF>')
    
    return ngrams

In [18]:
NGRAMS = padded_to_ngrams(PADDED, OHCO[:3], ngrams)

In [19]:
NGRAMS

w0        w1        w2
chap_num para_num sent_num token_num                              
1        0        0        0               <s>        to       mrs
                           1                to       mrs      </s>
                           2               mrs      </s>      None
                           3              </s>      None      None
                  1        0               <s>   saville   england
...                                        ...       ...       ...
28       82       1        11               in  darkness       and
                           12         darkness       and  distance
                           13              and  distance      </s>
                           14         distance      </s>      None
                           15             </s>      None      None

[85654 rows x 3 columns]

## Generate models

In [20]:
def ngrams_to_models(ngrams):
    global widx
    n = len(ngrams.columns)
    model = [None for i in range(n)]
    for i in range(n):
        if i == 0:
            model[i] = ngrams.value_counts('w0').to_frame('n')
            model[i]['p'] = model[i].n / model[i].n.sum()
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams.value_counts(widx[:i+1]).to_frame('n')    
            model[i]['cp'] = model[i].n / model[i-1].n
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
    return model

In [21]:
M = ngrams_to_models(NGRAMS)

## Define additional funcitons

In [22]:
def sentence_to_token(sent_list, file=True):
    
    # Convert list of sentences to dataframe
    if file:
        S = pd.read_csv("test_sentences_HW.txt", header=None, names=['sent_str'])
    else:
        S = pd.DataFrame(sent_list, columns=['sent_str'])
    S.index.name = 'sent_num'
    
    # Convert dataframe of sentences to TOKEN with normalized terms
    K = S.sent_str.apply(lambda x: pd.Series(x.split())).stack().to_frame('token_str')
    K['term_str'] = K.token_str.str.replace(r"[\W_]+", "", regex=True).str.lower()
    K.index.names = ['sent_num', 'token_num']
    
    return S, K

In [23]:
def test_model(model, ngrams, sents):
    
    global widx
    
    assert len(model) == len(ngrams.columns)
    
    n = len(model)
    ohco = ngrams.index.names
    
    R = []
    for i in range(n):
        T = ngrams.merge(M[i], on=widx[:i+1], how='left')
        T.index = ngrams.index
        T = T.reset_index().set_index(ohco + widx).i #.to_frame(f"i{i}")
        
        # This how we handle unseen combos
        T[T.isna()] = T.max()
        R.append(T.to_frame(f"i{i}"))
                
    return pd.concat(R, axis=1)

In [24]:
def compute_perplexity(results, test_sents, n=3):
    for i in range(n):
        test_sents[f"pp{i}"] = np.exp2(results.groupby('sent_num')[f"i{i}"].mean())
    return test_sents

In [25]:
def generate_text(M, n=250):
    
    if len(M) < 3:
        raise ValueError("Must have trigram model generated.")
    
    # Start list of words
    first_word = M[1].loc['<s>'].sample(weights='cp').index[0]
    
    words = ['<s>', first_word]
    
    for i in range(n):
        
        bg = tuple(words[-2:])

        # Try trigram model
        try:
            next_word = M[2].loc[bg].sample(weights='cp').index[0]

        # If not found in model, back off ...
        except KeyError as e1:
            try:
                # Get the last word in the bigram
                ug = bg[1]
                next_word = M[1].loc[ug].sample(weights='cp').index[0]
            
            except KeyError as e2:
                next_word = M[0].sample(weights='p').index[0]
                
        words.append(next_word)
    
    
    text = ' '.join(words[2:])
    print('\n\n'.join([str(i+1) + ' ' + line.replace('<s>','').strip().upper() for i, line in enumerate(text.split('</s>'))]))

# Question 1
List six words that precede the word "monster," excluding stop words (and sentence boundary markers). Stop words include 'a', 'an', 'the', 'this', 'that', etc. Hint: use the df.query() method.

In [26]:
filtered_df = NGRAMS.query('(w1 == "monster") & (w0 != "an") & (w0 != "a") & (w0 != "the") & (w0 != "this") & (w0 != "that") & (w0 != "<s>")')
filtered_df

w0       w1     w2
chap_num para_num sent_num token_num                            
9        3        17       25          miserable  monster   whom
14       8        0        1            abhorred  monster   </s>
19       25       4        23         detestable  monster   </s>
20       28       0        1             hideous  monster   </s>
28       4        9        5             hellish  monster  drink
         17       6        2            gigantic  monster   they

# Question 2
List the following sentences in ascending order of bigram perplexity according to the language model generated from the text:

In [27]:
TEST_SENTS, TEST_TOKENS = sentence_to_token("test_sentences_HW.txt")
TEST_PADDED = token_to_padded(TEST_TOKENS)
TEST_NGRAMS = padded_to_ngrams(TEST_PADDED, 'sent_num', ngrams)
R = test_model(M,TEST_NGRAMS, TEST_SENTS)
PP = compute_perplexity(R, TEST_SENTS)
PP.sort_values(by='pp1', ascending=True, inplace=True)
PP

,sent_str,pp0,pp1,pp2
sent_num,,,,
0,The monster is on the ice.,116.146797,80.632951,68.983256
3,He never knew the love of a family.,170.855904,136.870520,64.734928
2,I have never seen the aurora borealis.,340.954187,138.718691,81.279212
1,Flowers are happy things.,587.205060,533.982028,182.500000


# Question 3
Using the bigram model represented as a matrix, explore the relationship between bigram pairs using the following lists:

['he','she'] to select the indices.

['said','heard'] to select the columns.

In [28]:
#Filter bigram model to W0 W1 == he/she and W1 == said/heard
filtered_df = M[1].query('((w0 == "he") | (w0 == "she")) & ((w1 == "said") | (w1 == "heard"))')
filtered_df.unstack()

n             cp                   i          
w1  heard said     heard      said     heard      said
w0                                                    
he      5   21  0.008210  0.034483  6.928370  4.857981
she     3    3  0.011765  0.011765  6.409391  6.409391

# Question 4
Generate 20 sentences using the generate_text() function. Display the results.

In [29]:
generate_text(M)

1 EXCLAIMED

2 SUBJECT

3 WAS THE WORK OF INCONCEIVABLE DIFFICULTY AND LABOUR

4 THE MAJESTIC OAKS THE QUANTITY OF GAME AND THE SPRINGING OF A GUIDE

5 AND CONQUERED AND WHO ARE SO EMINENTLY DESERVING

6 WHICH HAD BECOME

7 AND COMFORTLESS SKY

8 TO RESTORE IT TO MY TOO KEEN SENSATIONS

9 THE DECAYING FRAME OF THE INHERITANCE OF ELIZABETH OR MY FATHER ENTERED THE HUT

10 DURING MY FIRST RESOLUTION WAS TO CONVEY ME AWAY AND WE RESOLVED TO REMAIN IN A SCENE OF THE GRAVE WORMS CRAWLING IN THE HEAVENS AND GAVE YOU AN IDEA STRUCK ME AS A DEFORMED AND HORRIBLE AS MYSELF

11 THOUSAND FEARS

12 FROM JUSTINE COULD DISSIPATE

13 SECRET

14 THAN A ROCK WHOSE HIGH SIDES WERE CONTINUALLY BEATEN UPON BY THE MODERN MASTERS PROMISE VERY LITTLE AND CONVERSED IN BROKEN ACCENTS WHILST I COMPREHENDED AND COULD SUBSIST UPON COARSER DIET

15 JUST RETRIBUTION BURNING WITHIN MY HANDS

16 OF MY FRIENDS AND MY FATHER S CONSENT TO GO AND HEAR THAT NO ASSISTANCE COULD SAVE ANY PART OF EACH DAY AT NOON WHEN I HEAR

# Question 5
Compute the redundancy $R$ for each of the n-gram models using the MLE of the joint probability of each ngram type. In other words, for each model, just use the .mle feature as $p$ in computing $H = \sum p(ng)log_2(1/p(ng))$. 

Hint: Remember that $R = 1 - \frac{H}{H_{max}}$, where $H$ is the actual entropy of the model and $H_{max}$ its maximum entropy.

In [30]:
#Calculate probability
M[1]['p'] = M[1].n /  M[1].n.sum()
M[2]['p'] = M[2].n /  M[2].n.sum()

#Calculate infomration
M[1]['log2p_1_over_p'] = np.log2(1/M[1].p)
M[2]['log2p_1_over_p'] = np.log2(1/M[2].p)

#Calculate N
M[1]['N'] = len(M[0].index)**2
M[2]['N'] = len(M[0].index)**3

#Log2(1/N)
M[1]['log2_1_over_N'] = np.log2(1/M[1].n.sum())
M[2]['log2_1_over_N'] = np.log2(1/M[2].n.sum())

In [31]:
sum(M[1].p*M[1].log2p_1_over_p)

14.130511267844236

In [32]:
sum(1/M[1].n.sum()*M[1].log2_1_over_N)

-8.258369986141371

In [33]:
#Calculate entropy (H) and max entropy (H_max)
M[1]['H'] = sum(M[1].p*M[1].log2p_1_over_p)
M[1]['H_max'] = sum(1/M[1].n.sum()*M[1].log2_1_over_N)
M[2]['H'] = sum(M[2].p*M[2].log2p_1_over_p)
M[2]['H_max'] = sum(1/M[2].n.sum()*M[2].log2_1_over_N)

In [34]:
#Calculate R
R_1 = 1-(M[1]['H']/M[1]['H_max'])
R_2 = 1-(M[2]['H']/M[2]['H_max'])

Does $R$ increase, decrease, or remain the same as the choice of n-gram increases in length? 

In [35]:
np.unique(R_1)

array([2.71105331])

In [36]:
np.unique(R_2)

array([2.13406816])

As shown above, $R$ decreases as the n-gram increases in length.